In [2]:
import numpy as np
import math
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
from odlib import *

In [41]:
def formatInputInfo(file:str):
    info=np.loadtxt(file,dtype=str,delimiter=",")
    time=np.array([float(info[i,0]) for i in range(1,len(info))])
    timestamps=np.copy(info[1:,1])
    return time,np.array([([float(info[i][2]),float(info[i][3]),float(info[i][4])], 
                    [float(info[i][5]),float(info[i][6]),float(info[i][7])]) for i in range(1,len(info))]), timestamps

def formatResultInfo():
    info=np.loadtxt("/home/soonali/Documents/ODCode/TestResults.txt",dtype=str,delimiter=",")
    return np.array([[float(info[i][j]) for j in range(2,14)] for i in range(1,len(info))])

In [42]:
class OD:
    '''Class that performs all orbital determination calculations'''
    
    def __init__(self):
        """ Initializes OD class
            Args:
                None
            Returns:
                None
        """
        # constants
        self.k = 0.0172020989484 #Gaussian gravitational constant  
        self.cAU = 173.144643267 #speed of light in au/(mean solar)day  
        self.mu = 1
        self.eps = np.radians(23.4374) #Earth's obliquity
        self.cAU = 173.144643267 # speed of light in au/(mean solar)day 
        
        self.data = Data()
        
    def genElements(self, pos:list, vel:list, time:float, update:bool=True):
        """ Calculates and returns the orbital elements given position, velocity, time
            Args:
                pos (list): the position vector
                vel (list): the velocity vector
                time (float): the time in Julian days
                update (bool): if True keeps the newly calculated orbital elements
            Returns:
                floats: the orbital elements; a,e,i,o,v,w,T,M
        """
        if update:
            self.pos,self.vel,self.time=pos,vel,time
            self.od=ODElements(self.pos,self.vel,self.time)
            self.a,self.e,self.i,self.o,self.v,self.w,self.T,self.M = self.od.getElements()
            return self.getElements()
        else:
            od=ODElements(pos,vel,time)
            return od.getElements()
    
    def getElements(self):
        """ Returns the orbital elements (already calculated)
            Args:
                rad (bool): True if return in radians
            Returns:
                floats: a,e,i,o,v,w,T,M
        """
        return self.a, self.e, self.i, self.o, self.v, self.w, self.T, self.M
    
    def printODElementsErr(self, res:str):
        """ Prints the OD elements and compares them to results (expected, calculated, % error)
            Args:
                res (list): results
            Returns:
                None
        """
        self.od.printError(res)
    
    def getT(self)->float:
        """ Returns the time of perihelion
            Args:
                None
            Returns:
                float: time of perihelion
        """
        return self.T # time of perihelion
    
    def SEL(self, taus:list, sunR2:list, rhohat2:list, coefD:list):
        """ Scalar Equation of Lagrange to calculate the roots (r) and rhos corresponding to each r
            Args:
                taus (list): a list of floats of taus [T1,T3,T]
                sunR2 (list): a list representing the sun vector R2
                rhohat2 (list): a list representing the rhohat2 vector
                coefD (list): a list of D coefficients [D0,D21,D22,D23]
            Returns:
                lists: roots (r's), rhos
        """
        T1,T3,T=taus[0],taus[1],taus[2]
        D0,D21,D22,D23=coefD[0],coefD[1],coefD[2],coefD[3]
        A1=T3/T
        B1=A1/6*(T**2-T3**2)
        A3=-T1/T
        B3=A3/6*(T**2-T1**2)
        A=(A1*D21-D22+A3*D23)/(-D0)
        B=(B1*D21+B3*D23)/(-D0)
        
        E=-2*(dot(rhohat2, sunR2))
        F=dot(sunR2,sunR2)
        
        a=-(A**2+A*E+F)
        b=-self.od.mu*(2*A*B+B*E)
        c=-self.od.mu**2*B**2
        
        coef=[c,0,0,b,0,0,a,0,1]#[1,0,a,0,0,b,0,0,c]
        res=poly.polyroots(coef)

        roots=[]
        for val in res: 
            if np.isreal(val) and np.real(val)>0: roots.append(np.real(val))

        rhos=[A+B/roots[i]**3 for i in range(len(roots))]
        return roots,rhos
    
    def ephemeris(self, time:float, date:str):
        """ Calculates RA and Dec given time and date, using previously calculated orbital elements
            Args:
                time (float): time to determine ephemeris for in Julian Days
                date (str): date to determine ephemeris for
            Returns:
                floats: ra, dec
        """
        n=self.k*math.sqrt(self.mu/(self.a**3))
        M=n*(time-self.T)
        E=newton(lambda E:M - E + self.e*np.sin(E), lambda E: -1+self.e*np.cos(E), M, 1e-14)
        
        pos=np.array([self.a*math.cos(E)-self.a*self.e, self.a*math.sqrt(1-self.e**2)*math.sin(E), 0])
        
        # the four rotations
        pos=rotZX(pos,np.deg2rad(self.w),np.deg2rad(self.i))
        pos=rotZX(pos,np.deg2rad(self.o),self.eps)
        
        rho=pos+self.getSunPos(date)
        rhohat=rho/getMag(rho)
        
        dec=math.asin(rhohat[2])
        cra=rhohat[0]/math.cos(dec)
        sra=rhohat[1]/math.cos(dec)

        ra=getAngle(sra,cra)
        
        dec=np.rad2deg(dec)
        ra=np.rad2deg(ra)
        
        dec=DECdecimalToDMS(dec)
        ra=RAdecimalToHMS(ra)
        
        return ra,dec
        
    def getSunPos(self, date:str, file:str)->list:
        """ Gets the vector from the Earth to the Sun given the date
            Args:
                date (str): the date to use
            Returns:
                list: sun vector R
        """
        info=np.loadtxt("/home/soonali/Documents/SunPos.txt",dtype=str,delimiter=",")

        timestamps=info[:,1]
        for i in range(len(timestamps)):
            if date in timestamps[i]: 
                line = i
                break
       
        stuff=info[line,:]
        x,y,z=float(stuff[2]),float(stuff[3]),float(stuff[4])
        return np.array([x,y,z])
    
    def getSunMag(self,date:str)->float:
        """ Gets the magnitude of the position vector from sun to earth
            Args:
                date (str): the date to use
            Returns:
                float: the magnitude of vector R
        """
        return getMag(getSunPos(date))
        
    def fg(self, tau:float, r2:list, r2dot:list, flag:bool):
        """ Gets the f and g values given one time
            Args:
                tau (float): the time in Julian Days
                r2 (list): the position vector 2
                r2dot (list): the velocity vector 2
                flag (bool): True if fourth term of Taylor Series expansion is needed
            Returns:
                floats: f, g
        """
        u=self.mu/getMag(r2)**3
        z=dot(r2,r2dot)/(dot(r2,r2))
        q=dot(r2dot,r2dot)/(dot(r2,r2))-u
        
        f=1-1/2*u*tau**2+1/2*u*z*tau**3
        g=tau-1/6*u*tau**3
        if flag:
            f+=1/24*(3*u*q-15*u*z**2+u**2)*tau**4
            g+=1/4*u*z*tau**4
        
        return f, g
        
    def getFGVals(self, tau1:float, tau3:float, r2:list, r2dot:list, flag1:bool, flag3:bool):
        """ Gets all f and g values
            Args:
                tau1 (float): the time in Julian Days for observation 1
                tau3 (float): the time in Julian days for observation 3
                r2 (list): the position vector 2
                r2dot (list): the velocity vector 2
                flag1 (bool): True if fourth term of Taylor Series expansion is needed for observation1
                flag2 (bool): True if fourth term of Taylor Series expansion is needed for observation2
            Returns:
                lists: [f1,f3], [g1,g3]
        """
        f1,g1=self.fg(tau1,r2,r2dot,flag1)
        f3,g3=self.fg(tau3,r2,r2dot,flag3)
        return [f1,f3], [g1,g3]
    
    def getDCoef(self, ra:list, dec:list, R1:list, R2:list, R3:list)->list:
        """ Gets the D coefficients given the ra and dec for three observations (in radians)
            Args:
                ra (list): the right ascensions for three observations (radians)
                dec (list): the declinations for three observations (radians)
                R1 (list): the sun vector for observation 1
                R2 (list): the sun vector for observation 2
                R3 (list): the sun vector for observation 3
            Returns:
                list: [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33]
        """
        ra1,ra2,ra3=ra[0],ra[1],ra[2]
        dec1,dec2,dec3=dec[0],dec[1],dec[2]
        
        rhohat1=np.array([np.cos(ra1)*np.cos(dec1), np.sin(ra1)*np.cos(dec1), np.sin(dec1)])
        rhohat2=np.array([np.cos(ra2)*np.cos(dec2), np.sin(ra2)*np.cos(dec2), np.sin(dec2)])
        rhohat3=np.array([np.cos(ra3)*np.cos(dec3), np.sin(ra3)*np.cos(dec3), np.sin(dec3)])
        
        D0=dot(rhohat1, cross(rhohat2,rhohat3))
        D11=dot(cross(R1, rhohat2),rhohat3)
        D12=dot(cross(R2, rhohat2),rhohat3)
        D13=dot(cross(R3, rhohat2),rhohat3)
        
        D21=dot(cross(rhohat1,R1), rhohat3)
        D22=dot(cross(rhohat1,R2), rhohat3)
        D23=dot(cross(rhohat1,R3), rhohat3)
        
        D31=dot(rhohat1, cross(rhohat2,R1))
        D32=dot(rhohat1, cross(rhohat2,R2))
        D33=dot(rhohat1, cross(rhohat2,R3))
     
        return [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33]
        
        

    

In [43]:
file = "/home/soonali/Documents/ODCode/TestInput.txt"
times,data,timestamps=formatInputInfo(file)
date = '2018-Jul-14 00:00:00'
#time = 2458313.5000000    
line = 0
for i in range(len(timestamps)):
    if date in timestamps[i]: 
        line = i
        break

time,info=times[line],data[line]
pos,vel=info[0],info[1]
od=OD()
od.genElements(pos,vel,time)

# testing OD elements class
def testODElements(od):
    results=formatResultInfo()[24]
    od.printODElementsErr(results)
    
testODElements(od)

Semi-major axis: 1.056800055744403 1.0568000556596946 8.015561884787836e-09
Eccentricity: 0.3442331093278357 0.3442331094157307 2.5533576582325764e-08
Inclination: 25.1552516656358 25.155251665637707 7.584131023113933e-12
Longitude of Ascending Node: 236.2379806531768 236.23798065316336 5.690646577409404e-12
True anomaly: 158.9559248709949 158.9559248763771 3.385982069686107e-09
Argument of perihelion: 255.5046142809498 255.5046142714118 3.732998971925637e-09
Time of Perihelion Passage T: 2458158.720849546 2458158.720849547 3.7886999188303465e-14
Mean Anomaly: 140.4194576216498 140.41945762513234 2.4800982265579533e-09


In [44]:
def testFG(od):
    tau1 = -0.32618617484601165  
    tau3 = 0.0508408854033231  
    r2 = [0.26799552002875776, -1.3726277901924608, -0.5026729612047128]  
    r2dot = [0.8456809141954584, -0.3838382184712308, 0.14215854191172816] 

    f,g=od.getFGVals(tau1,tau3,r2,r2dot,False,False)
    f1,f3=f[0],f[1]
    g1,g3=g[0],g[1]

    print("Test case 2:")
    print("f1:",f1,error(0.9821596284506794,f1))
    print("f3:",f3,error(0.9996124342607986,f3))
    print("g1:",g1,error(-0.32442392608030396,g1))
    print("g3:",g3,error(0.05083421257607972,g3))

    tau1 = -0.3261857571141891  
    tau3 = 0.05084081855693949  
    r2 = [0.26662393644794813, -1.381475976476564, -0.5048589337503169]  
    r2dot = [0.8442117090940343, -0.39728396707075087, 0.14202728258915864]  

    f,g=od.getFGVals(tau1,tau3,r2,r2dot,True,True)
    f1,f3=f[0],f[1]
    g1,g3=g[0],g[1]

    print("\nTest case 3:")
    print("f1:",f1,error(0.9823149707782799  ,f1))
    print("f3:",f3,error(0.99961917185922  ,f3))
    print("g1:",g1,error(-0.32418770657924106  ,g1))
    print("g3:",g3,error(0.05083441832100904,g3))
    
testFG(od)


Test case 2:
f1: 0.9821596284506794 0.0
f3: 0.9996124342607986 0.0
g1: -0.32442392608030396 0.0
g3: 0.05083421257607972 0.0

Test case 3:
f1: 0.9823149707782799 0.0
f3: 0.99961917185922 0.0
g1: -0.32418770657924106 0.0
g3: 0.05083441832100904 0.0


In [45]:
def testEphemeris(od):
    newdate = '2018-Aug-03 00:00:00'
    newjdtime = 2458333.5000000
    ra,dec=od.ephemeris(newjdtime, newdate)
    
    # for jul 14 00:00:00
    #ra,dec=od.ephemeris(jdtime, date)
    #print("\n% error ra:", error(HMStoDeg(18,14,30.53),HMStoDeg(ra[0],ra[1],ra[2])))
    #print("% error dec:", error(DMStoDeg(36,9,46.0),DMStoDeg(dec[0],dec[1],dec[2])))

    #print("\nexpected:")
    #print("ra: 18 14 30.53")
    #print("dec: +36 09 46.0")
    
    print("calculated:")
    print("ra:", ra)
    print("dec:",dec)
    
    print("\n% error ra:", error(HMStoDeg(17,42,21.12),HMStoDeg(ra[0],ra[1],ra[2])))
    print("% error dec:", error(DMStoDeg(31,52,26.8),DMStoDeg(dec[0],dec[1],dec[2])))

    print("\nexpected:")
    print("ra: 17 42 21.12")
    print("dec: +31 52 26.8")
    
testEphemeris(od)

OSError: /home/soonali/Documents/SunPos.txt not found.

In [30]:
def testDCoefs(od):
    """2021 06 25 00:00:00.000 18:25:08.44 -17:26:41.3 -5.985728598861461E-02 9.309676159065817E-01 4.035414693476737E-01 
 
    2021 07 05 00:00:00.000 18:15:28.85 -16:27:16.5 -2.271502585826002E-01 9.092709064712199E-01 3.941342306093848E-01 
 
    2021 07 15 00:00:00.000 18:05:40.89 -15:30:48.9 -3.881336047533506E-01 8.619617590425438E-01 3.736284118981542E-01"""
    
    ra=[HMStoDeg(18,25,8.44, convert=True),HMStoDeg(18,15,28.85, convert=True),HMStoDeg(18,5,40.89, convert=True)]
    dec=[DMStoDeg(-17,26,41.3,convert=True), DMStoDeg(-16,27,16.5,convert=True), DMStoDeg(-15,30,48.9,convert=True)]
    R1=[-5.985728598861461E-02,9.309676159065817E-01,4.035414693476737E-01]
    R2=[-2.271502585826002E-01,9.092709064712199E-01,3.941342306093848E-01]
    R3=[-3.881336047533506E-01,8.619617590425438E-01,3.736284118981542E-01]

    dCoefs=od.getDCoef(ra,dec,R1,R2,R3)
    print(dCoefs)

    ans=[7.153072967571068e-05,-0.0051069590315919065,-0.0021329782464551625,
     0.000902451104005289,0.010014747188866925, 0.0038526558958926135, -0.0024208093416812626, -0.004989761292019754, 
     -0.001794759627912354, 0.0014522846235393017]

    for i in range(10):
        print("error:", error(dCoefs[i],ans[i]))

testDCoefs(od)

[7.15307296756968e-05, -0.005106959031591908, -0.0021329782464551555, 0.000902451104005289, 0.01001474718886704, 0.0038526558958927246, -0.0024208093416811793, -0.0049897612920198585, -0.0017947596279124511, 0.0014522846235392184]
error: 1.9401155098980568e-11
error: 3.396783614761191e-14
error: 3.253147994096579e-13
error: 0.0
error: 1.1432315489875251e-12
error: 2.881708241342689e-12
error: 3.4396234933999593e-12
error: 2.085939636532701e-12
error: 5.412675499486995e-12
error: 5.7334991707042714e-12
